# Summorize Pharma General and create xlsx-Files

In [1]:
import pandas as pd
import mysql.connector

## Connect

In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [3]:
df_list_raw = pd.read_sql("""SELECT 
    sum(tra_value) as sum_value,
    trc_name as category,
    pha_name as pharma,
    tra_year as year,
    rec_type as type
  FROM transaction
  LEFT JOIN pharma ON tra_fk_pharma = pha_id
  LEFT JOIN transaction_category on tra_fk_transaction_category = trc_id
  LEFT JOIN recipient ON rec_id = tra_fk_recipient
  GROUP BY tra_fk_pharma, tra_year, tra_fk_transaction_category, rec_type""", con=db_connection)

df_accu_raw = pd.read_sql("""SELECT 
    sum(acc_value) as sum_value,
    trc_name as category,
    pha_name as pharma,
    acc_year as year,
    acc_type as type
  FROM accumulation
  LEFT JOIN pharma ON acc_fk_pharma = pha_id
  LEFT JOIN transaction_category on acc_fk_transaction_category = trc_id
  GROUP BY acc_fk_pharma, acc_year, acc_fk_transaction_category, acc_type""", con=db_connection)


## Export

In [34]:
df_list = df_list_raw.copy()
df_accu = df_accu_raw.copy()

#Concat
df_data = pd.concat([df_list, df_accu], sort=False)

df_group = df_data.groupby(['pharma', 'year']).agg({'sum_value': 'sum'}).reset_index()

df_pivot = pd.pivot_table(df_group, columns='year', index='pharma', values='sum_value').reset_index()

#Fillna
#df_pivot = df_pivot.fillna(0)

#Total
df_pivot['total'] = df_pivot.sum(axis=1)

df_pivot = df_pivot.sort_values('total', ascending=False)

#Write to Excel
writer = pd.ExcelWriter('../../data/99. analyzes/excel/pharmas/pharmas_overview.xlsx', options={'encoding':'utf-8'})

df_pivot.to_excel(writer, 'overview', index=False)

writer.save()